# df_points with UNIX Timestamp #
## Creates df_points.csv with all points and idletimes #
## Checks the valid geo location
## deletes the invalid points: 

2018 - invalides:   41743 from 139171 that is 29.994036113845556 %

2019 - invalides:   52523 from 550887 that is 9.534260202183024 % - 05:40:19

2020 - invalides:   24163 from 539064 that is 4.482399121440126 % - 05:40:06
                    18086 from 662386 that is 2.7304321045432722 %

2021 - invalides:   21382 from 552253 that is 3.8717761605640897 % - 05:51:53

### ca 1h for 1 year

In [1]:
def import_data(source):
    return pd.read_csv(source)

In [2]:
def dic_out_of_df(df01):
    import pandas as pd
    from tqdm import tqdm
    # Dict out of DataFrames of Trips per Bike

    # init DFs for every bike
    myDFs = {}

    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] not in myDFs:
            myDFs[df01['bike_id'][ind]] = pd.DataFrame(
                columns=['bike_id',
                         'start_time',
                         'end_time',
                         'start_lat',
                         'start_lng',
                         'end_lat',
                         'end_lng',
                         'end_station_number'])
    return myDFs

In [3]:
def add_rentals_to_df(df01,myDFs):
    from tqdm import tqdm
    # Add all rentals to their bike_id DF
    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] in myDFs:
            myDFs[df01['bike_id'][ind]].loc[df01.index[ind]] = df01.iloc[ind]
    return myDFs

In [4]:
def sort_dfs(myDFs):
    # sort every DF
    for df in myDFs.values():
        df[['bike_id', 'start_time','end_time']] = df[['bike_id', 'start_time','end_time']].astype(int)
        df[['start_lat','start_lng', 'end_lat', 'end_lng']] = df[['start_lat','start_lng', 'end_lat', 'end_lng']].astype(float)
        df.sort_values(by=['start_time'], inplace=True)
        df.reset_index(drop=True, inplace=True)
    return myDFs

In [5]:
# returns distance in meter
# source: https://www.it-swarm.com.de/de/python/wie-kann-ich-die-entfernung-zwischen-zwei-punkten-breitengrad-laengengrad-schnell-schaetzen/1072488907/
def get_distance(Lat1, Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1)*cos((Lat2 + Lat1)*0.00872664626)
    return 111.138*sqrt(x*x+y*y)*1000

In [6]:
def create_df_points(myDFs,radius):
    # DF of all points with stoodtime
    in_val = 0
    val = 0 
    
    df_points = pd.DataFrame(columns=['bike_id', 'lat', 'lng', 'idle_time', 'time_start', 'time_end', 'end_station_number'])
    #df_points[['bike_id', 'idle_time', 'time_start', 'time_end']] = df_points[['bike_id', 'idle_time', 'time_start', 'time_end']].astype(int)
    #df_points[['lng', 'lat']] = df_points[['lng', 'lat']].astype(float)
    
    for df in tqdm(myDFs.values()):
        for ind in df.index[1:]:
            #in SECOUNDS
            _idle_time = (df['start_time'][ind] - df['end_time'][ind - 1]).astype(int)
            distance = get_distance(df['end_lat'][ind - 1], df['end_lng'][ind - 1], df['start_lat'][ind], df['start_lng'][ind])
            if distance > radius:
                in_val +=1
                continue
            val +=1
            dict = {'bike_id': df['bike_id'][ind], 'lng': df['start_lng'][ind], 'lat': df['start_lat'][ind],
                    'idle_time': _idle_time, 'time_start': df['end_time'][ind - 1], 'time_end': df['start_time'][ind], 'end_station_number': df['end_station_number'][ind-1]}
            df_points = df_points.append(dict, ignore_index=True)
    print(f"invalides: {in_val} from {val} that is {in_val/val * 100} %")
    return df_points

In [7]:
def keplerMap(df_points):
    heatmap = KeplerGl()
    heatmap.add_data(data=df_points, name='points')
    heatmap.save_to_html(file_name='heatmap_test.html')

# Main 
## Create map

In [8]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
import time

ts1 = time.time()

source = '../data/raw_lendings/raw_lendings_2020.csv'

df = import_data(source)
df.tail()

,bike_id,start_time,end_time,start_lat,start_lng,end_lat,end_lng,end_station_number
681701,72175,1602481901,1602482574,51.322419,12.335329,51.339422,12.354404,NaN
681702,93568,1577978192,1577978750,51.333676,12.373900,51.342431,12.360998,NaN
681703,72079,1589732284,1589735621,51.338262,12.388078,51.358609,12.359677,NaN
681704,71968,1598182493,1598182867,51.339439,12.395017,51.345320,12.406171,NaN
681705,93446,1598542177,1598543426,0.000000,0.000000,51.330160,12.405212,NaN


In [9]:
myDfs = dic_out_of_df(df)
myDfs = add_rentals_to_df(df, myDfs)
myDfs = sort_dfs(myDfs)


100%|██████████| 681706/681706 [15:15<00:00, 744.59it/s]


# Create DF_points
## without invalid points

In [10]:
from math import cos, sqrt
from tqdm import tqdm

radius = 20
df_points = create_df_points(myDfs,radius)

100%|██████████| 1234/1234 [1:10:27<00:00,  3.43s/it]

invalides: 18086 from 662386 that is 2.7304321045432722 %


In [11]:
df_points.tail()

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number
662381,89122.0,51.305600,12.369659,2460.0,1.608044e+09,1.608047e+09,NaN
662382,89122.0,51.306316,12.382508,179252.0,1.608047e+09,1.608226e+09,NaN
662383,89122.0,51.306662,12.380719,50284.0,1.608229e+09,1.608279e+09,NaN
662384,89122.0,51.306698,12.380143,1760.0,1.608280e+09,1.608281e+09,NaN
662385,89122.0,51.350196,12.308767,460964.0,1.608890e+09,1.609351e+09,NaN


# Split Points at midnight

## save feature idle_time_next_day

In [12]:
df_points['idle_time'] = pd.to_timedelta(df_points['idle_time'],unit='s')
df_points['time_start'] = pd.to_datetime(df_points['time_start'],unit='s')
df_points['time_end'] = pd.to_datetime(df_points['time_end'],unit='s')

In [13]:
from tqdm import tqdm
from datetime import timedelta
import numpy as np

changed = 1
iterations = 0

df_points['idle_time_next_day'] = np.nan
df_points['over_night'] = 0

while changed:
    iterations += 1
    changed = 0
    for ind in tqdm(df_points.index):
        ts = df_points['time_start'][ind]
        te = df_points['time_end'][ind]
        t0 = ts.replace(hour=23, minute=59, second=59)
        
        # is timestood greater than same date 23:59:59
        #   -> over midnight
        if ts + df_points['idle_time'][ind] > t0:
            changed = 1
            
            underhang = t0 - ts
            overhang = te - t0
            
            # fix current day
            df_points['time_end'][ind] = ts.replace(hour=23, minute=59, second=59)
            df_points['idle_time'][ind] = underhang
            df_points['over_night'][ind] = 1
            df_points['idle_time_next_day'][ind] = overhang
            
            # add new row for the next day
            time_start = ts.replace(hour=0, minute=0, second=0)
            time_start += timedelta(days=1)
            flag = 0
            #if time_start + overhang > time_start.replace(hour=23, minute=59, second=59):
            #    flag = 1
            
            dict = {'bike_id': df_points['bike_id'][ind], 'lng': df_points['lng'][ind], 'lat': df_points['lat'][ind],
                    'idle_time': overhang, 'time_start': time_start, 'time_end': te,'over_night': flag, 'idle_time_next_day': np.nan, 'end_station_number': df_points['end_station_number'][ind]}
            df_points = df_points.append(dict, ignore_index=True)

print(iterations)

100%|██████████| 883321/883321 [00:40<00:00, 21838.47it/s]

158


In [14]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night
0,71955.0,51.370693,12.376540,0 days 00:22:17,2020-01-04 11:22:58,2020-01-04 11:45:15,NaN,NaN,0
1,71955.0,51.345639,12.379167,0 days 07:07:56,2020-01-04 12:01:02,2020-01-04 19:08:58,4055.0,NaN,0
2,71955.0,51.332422,12.339126,0 days 01:01:04,2020-01-04 19:25:48,2020-01-04 20:26:52,NaN,NaN,0
3,71955.0,51.325316,12.373336,0 days 00:24:11,2020-01-04 20:39:30,2020-01-04 21:03:41,NaN,NaN,0
4,71955.0,51.336462,12.374997,0 days 01:23:06,2020-01-04 21:09:39,2020-01-04 22:32:45,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
883316,93544.0,51.321160,12.269497,0 days 23:59:59,2020-07-21 00:00:00,2020-07-21 23:59:59,NaN,3 days 22:41:48,1
883317,93544.0,51.321160,12.269497,0 days 23:59:59,2020-07-22 00:00:00,2020-07-22 23:59:59,NaN,2 days 22:41:48,1
883318,93544.0,51.321160,12.269497,0 days 23:59:59,2020-07-23 00:00:00,2020-07-23 23:59:59,NaN,1 days 22:41:48,1
883319,93544.0,51.321160,12.269497,0 days 23:59:59,2020-07-24 00:00:00,2020-07-24 23:59:59,NaN,0 days 22:41:48,1


idle time in min

In [15]:
def idle_time_to_min(row):
    return row['idle_time'].seconds / 60

df_points['idle_time'] = df_points.apply(idle_time_to_min, axis=1)

add idle_time_next_day feature

In [16]:
df_points['idle_time_next_day'] = pd.to_timedelta(df_points['idle_time_next_day'])

def idle_time_next_day_to_min(row):
    if row['over_night'] == 1:
        t = row['idle_time_next_day'].total_seconds() / 60
        if t < 1439:
            return t
        else: return 1439
    else:
        return 0

df_points['idle_time_next_day'] = df_points.apply(idle_time_next_day_to_min, axis=1)


In [17]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night
0,71955.0,51.370693,12.376540,22.283333,2020-01-04 11:22:58,2020-01-04 11:45:15,NaN,0.0,0
1,71955.0,51.345639,12.379167,427.933333,2020-01-04 12:01:02,2020-01-04 19:08:58,4055.0,0.0,0
2,71955.0,51.332422,12.339126,61.066667,2020-01-04 19:25:48,2020-01-04 20:26:52,NaN,0.0,0
3,71955.0,51.325316,12.373336,24.183333,2020-01-04 20:39:30,2020-01-04 21:03:41,NaN,0.0,0
4,71955.0,51.336462,12.374997,83.100000,2020-01-04 21:09:39,2020-01-04 22:32:45,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...
883316,93544.0,51.321160,12.269497,1439.983333,2020-07-21 00:00:00,2020-07-21 23:59:59,NaN,1439.0,1
883317,93544.0,51.321160,12.269497,1439.983333,2020-07-22 00:00:00,2020-07-22 23:59:59,NaN,1439.0,1
883318,93544.0,51.321160,12.269497,1439.983333,2020-07-23 00:00:00,2020-07-23 23:59:59,NaN,1439.0,1
883319,93544.0,51.321160,12.269497,1439.983333,2020-07-24 00:00:00,2020-07-24 23:59:59,NaN,1361.8,1


change type to int

In [18]:
df_points['idle_time'] = df_points['idle_time'].astype(int)
df_points['idle_time_next_day'] = df_points['idle_time_next_day'].astype(int)
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night
0,71955.0,51.370693,12.376540,22,2020-01-04 11:22:58,2020-01-04 11:45:15,NaN,0,0
1,71955.0,51.345639,12.379167,427,2020-01-04 12:01:02,2020-01-04 19:08:58,4055.0,0,0
2,71955.0,51.332422,12.339126,61,2020-01-04 19:25:48,2020-01-04 20:26:52,NaN,0,0
3,71955.0,51.325316,12.373336,24,2020-01-04 20:39:30,2020-01-04 21:03:41,NaN,0,0
4,71955.0,51.336462,12.374997,83,2020-01-04 21:09:39,2020-01-04 22:32:45,NaN,0,0
...,...,...,...,...,...,...,...,...,...
883316,93544.0,51.321160,12.269497,1439,2020-07-21 00:00:00,2020-07-21 23:59:59,NaN,1439,1
883317,93544.0,51.321160,12.269497,1439,2020-07-22 00:00:00,2020-07-22 23:59:59,NaN,1439,1
883318,93544.0,51.321160,12.269497,1439,2020-07-23 00:00:00,2020-07-23 23:59:59,NaN,1439,1
883319,93544.0,51.321160,12.269497,1439,2020-07-24 00:00:00,2020-07-24 23:59:59,NaN,1361,1


# Save V2 df_points split

In [19]:
df_points['idle_time'].max()

1439

In [20]:
df_points.head()

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night
0,71955.0,51.370693,12.376540,22,2020-01-04 11:22:58,2020-01-04 11:45:15,NaN,0,0
1,71955.0,51.345639,12.379167,427,2020-01-04 12:01:02,2020-01-04 19:08:58,4055.0,0,0
2,71955.0,51.332422,12.339126,61,2020-01-04 19:25:48,2020-01-04 20:26:52,NaN,0,0
3,71955.0,51.325316,12.373336,24,2020-01-04 20:39:30,2020-01-04 21:03:41,NaN,0,0
4,71955.0,51.336462,12.374997,83,2020-01-04 21:09:39,2020-01-04 22:32:45,NaN,0,0


## Add H3 Index

In [21]:
import h3

df_points['hex_id'] = 0

for ind in tqdm(df_points.index):
    df_points['hex_id'][ind] = h3.geo_to_h3(df_points['lat'][ind], df_points['lng'][ind], 8)
df_points.head()

100%|██████████| 883321/883321 [00:16<00:00, 52137.56it/s]


,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night,hex_id
0,71955.0,51.370693,12.376540,22,2020-01-04 11:22:58,2020-01-04 11:45:15,NaN,0,0,881f1a8cd7fffff
1,71955.0,51.345639,12.379167,427,2020-01-04 12:01:02,2020-01-04 19:08:58,4055.0,0,0,881f1a8cb1fffff
2,71955.0,51.332422,12.339126,61,2020-01-04 19:25:48,2020-01-04 20:26:52,NaN,0,0,881f1a8d9bfffff
3,71955.0,51.325316,12.373336,24,2020-01-04 20:39:30,2020-01-04 21:03:41,NaN,0,0,881f1a1643fffff
4,71955.0,51.336462,12.374997,83,2020-01-04 21:09:39,2020-01-04 22:32:45,NaN,0,0,881f1a8cb5fffff


# Add weather Data

In [22]:
dfw = pd.read_csv('../data/weather_data.csv')
dfw['datetime'] = pd.to_datetime(dfw['dt'], unit='s')
dfw = dfw.set_index(['datetime'])
dfw.head()

df_points['temp'] = np.nan
df_points['rain'] = np.nan
df_points['snow'] = np.nan
df_points['wind_speed'] = np.nan
df_points['humidity'] = np.nan

from tqdm import tqdm
pd.options.mode.chained_assignment = None

for ind in tqdm(df_points.index):
    wint = df_points['time_start'][ind].replace(minute=0,second=0)
    df_points['temp'][ind] = dfw['temp'][wint].copy()
    df_points['rain'][ind] = dfw['rain_1h'][wint].copy()
    df_points['snow'][ind] = dfw['snow_1h'][wint].copy()
    df_points['wind_speed'][ind] = dfw['wind_speed'][wint].copy()
    df_points['humidity'][ind] = dfw['humidity'][wint].copy()

df_points

100%|██████████| 883321/883321 [20:04<00:00, 733.34it/s]


,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night,hex_id,temp,rain,snow,wind_speed,humidity
0,71955.0,51.370693,12.376540,22,2020-01-04 11:22:58,2020-01-04 11:45:15,NaN,0,0,881f1a8cd7fffff,4.88,0.11,NaN,0.89,76.0
1,71955.0,51.345639,12.379167,427,2020-01-04 12:01:02,2020-01-04 19:08:58,4055.0,0,0,881f1a8cb1fffff,5.39,NaN,NaN,0.89,75.0
2,71955.0,51.332422,12.339126,61,2020-01-04 19:25:48,2020-01-04 20:26:52,NaN,0,0,881f1a8d9bfffff,3.07,0.27,NaN,8.94,88.0
3,71955.0,51.325316,12.373336,24,2020-01-04 20:39:30,2020-01-04 21:03:41,NaN,0,0,881f1a1643fffff,2.98,NaN,NaN,7.15,87.0
4,71955.0,51.336462,12.374997,83,2020-01-04 21:09:39,2020-01-04 22:32:45,NaN,0,0,881f1a8cb5fffff,2.85,NaN,NaN,8.94,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883316,93544.0,51.321160,12.269497,1439,2020-07-21 00:00:00,2020-07-21 23:59:59,NaN,1439,1,881f1a8debfffff,15.89,NaN,NaN,2.24,72.0
883317,93544.0,51.321160,12.269497,1439,2020-07-22 00:00:00,2020-07-22 23:59:59,NaN,1439,1,881f1a8debfffff,13.39,NaN,NaN,2.10,62.0
883318,93544.0,51.321160,12.269497,1439,2020-07-23 00:00:00,2020-07-23 23:59:59,NaN,1439,1,881f1a8debfffff,15.08,NaN,NaN,0.89,67.0
883319,93544.0,51.321160,12.269497,1439,2020-07-24 00:00:00,2020-07-24 23:59:59,NaN,1361,1,881f1a8debfffff,15.34,NaN,NaN,3.58,66.0


# Fill NaNS

In [23]:
df_points['rain'] = df_points['rain'].fillna(0)
df_points['snow'] = df_points['snow'].fillna(0)
df_points['end_station_number'] = df_points['end_station_number'].fillna(0)

df_points['wind_speed'] = df_points['wind_speed'].fillna(df_points['wind_speed'].mean())
df_points['humidity'] = df_points['humidity'].fillna(df_points['humidity'].mean())
df_points = df_points.sort_values(by=['time_start']).reset_index(drop=True)

add unix timestamps

In [24]:
df_points['dt_start'] = pd.to_datetime(df_points['time_start']).map(pd.Timestamp.timestamp).astype(int)
df_points['dt_end'] = pd.to_datetime(df_points['time_end']).map(pd.Timestamp.timestamp).astype(int)
df_points['bike_id'] = df_points['bike_id'].astype(int)
df_points['end_station_number'] = df_points['end_station_number'].astype(int)

In [25]:
df_points


,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night,hex_id,temp,rain,snow,wind_speed,humidity,dt_start,dt_end
0,93513,51.317649,12.373610,50,2019-12-31 23:09:34,2019-12-31 23:59:59,0,1439,1,881f1a1647fffff,2.13,0.0,0.0,3.58,90.0,1577833774,1577836799
1,74459,51.320440,12.345386,27,2019-12-31 23:32:03,2019-12-31 23:59:59,0,98,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835123,1577836799
2,75845,51.320418,12.345038,27,2019-12-31 23:32:48,2019-12-31 23:59:59,0,99,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835168,1577836799
3,100227,51.339529,12.373269,16,2019-12-31 23:43:30,2019-12-31 23:59:59,0,133,1,881f1a8cb5fffff,2.13,0.0,0.0,3.58,90.0,1577835810,1577836799
4,73838,51.344711,12.405900,16,2019-12-31 23:43:47,2019-12-31 23:59:59,0,25,1,881f1ab965fffff,2.13,0.0,0.0,3.58,90.0,1577835827,1577836799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883316,71996,51.332053,12.403324,130,2020-12-30 20:41:15,2020-12-30 22:51:54,0,0,0,881f1a165bfffff,2.61,0.0,0.0,3.58,82.0,1609360875,1609368714
883317,93372,51.320791,12.405072,52,2020-12-30 20:46:14,2020-12-30 21:38:59,0,0,0,881f1a1657fffff,2.61,0.0,0.0,3.58,82.0,1609361174,1609364339
883318,41797,51.331062,12.390168,24,2020-12-30 20:48:59,2020-12-30 21:13:09,0,0,0,881f1a165dfffff,2.61,0.0,0.0,3.58,82.0,1609361339,1609362789
883319,93645,51.340196,12.402059,88,2020-12-30 21:00:57,2020-12-30 22:29:07,0,0,0,881f1ab965fffff,2.76,0.0,0.0,4.47,82.0,1609362057,1609367347


# Add start time Feature

In [26]:
def add_start_min_feature(row):
    return ((row['dt_start'] % 86400) / 60).__round__(0)

df_points['start_min'] = df_points.apply(add_start_min_feature,axis=1)
df_points['start_min'] = df_points['start_min'].astype(int)

In [27]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night,hex_id,temp,rain,snow,wind_speed,humidity,dt_start,dt_end,start_min
0,93513,51.317649,12.373610,50,2019-12-31 23:09:34,2019-12-31 23:59:59,0,1439,1,881f1a1647fffff,2.13,0.0,0.0,3.58,90.0,1577833774,1577836799,1390
1,74459,51.320440,12.345386,27,2019-12-31 23:32:03,2019-12-31 23:59:59,0,98,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835123,1577836799,1412
2,75845,51.320418,12.345038,27,2019-12-31 23:32:48,2019-12-31 23:59:59,0,99,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835168,1577836799,1413
3,100227,51.339529,12.373269,16,2019-12-31 23:43:30,2019-12-31 23:59:59,0,133,1,881f1a8cb5fffff,2.13,0.0,0.0,3.58,90.0,1577835810,1577836799,1424
4,73838,51.344711,12.405900,16,2019-12-31 23:43:47,2019-12-31 23:59:59,0,25,1,881f1ab965fffff,2.13,0.0,0.0,3.58,90.0,1577835827,1577836799,1424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883316,71996,51.332053,12.403324,130,2020-12-30 20:41:15,2020-12-30 22:51:54,0,0,0,881f1a165bfffff,2.61,0.0,0.0,3.58,82.0,1609360875,1609368714,1241
883317,93372,51.320791,12.405072,52,2020-12-30 20:46:14,2020-12-30 21:38:59,0,0,0,881f1a1657fffff,2.61,0.0,0.0,3.58,82.0,1609361174,1609364339,1246
883318,41797,51.331062,12.390168,24,2020-12-30 20:48:59,2020-12-30 21:13:09,0,0,0,881f1a165dfffff,2.61,0.0,0.0,3.58,82.0,1609361339,1609362789,1249
883319,93645,51.340196,12.402059,88,2020-12-30 21:00:57,2020-12-30 22:29:07,0,0,0,881f1ab965fffff,2.76,0.0,0.0,4.47,82.0,1609362057,1609367347,1261


# Add day Feature

In [28]:
df_points.dtypes

bike_id                        int64
lat                          float64
lng                          float64
idle_time                      int64
time_start            datetime64[ns]
time_end              datetime64[ns]
end_station_number             int64
idle_time_next_day             int64
over_night                     int64
hex_id                        object
temp                         float64
rain                         float64
snow                         float64
wind_speed                   float64
humidity                     float64
dt_start                       int64
dt_end                         int64
start_min                      int64
dtype: object

In [29]:
def add_day_feature(row):
    return row['time_start'].dayofweek

df_points['day'] = df_points.apply(add_day_feature,axis=1)

In [30]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night,hex_id,temp,rain,snow,wind_speed,humidity,dt_start,dt_end,start_min,day
0,93513,51.317649,12.373610,50,2019-12-31 23:09:34,2019-12-31 23:59:59,0,1439,1,881f1a1647fffff,2.13,0.0,0.0,3.58,90.0,1577833774,1577836799,1390,1
1,74459,51.320440,12.345386,27,2019-12-31 23:32:03,2019-12-31 23:59:59,0,98,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835123,1577836799,1412,1
2,75845,51.320418,12.345038,27,2019-12-31 23:32:48,2019-12-31 23:59:59,0,99,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835168,1577836799,1413,1
3,100227,51.339529,12.373269,16,2019-12-31 23:43:30,2019-12-31 23:59:59,0,133,1,881f1a8cb5fffff,2.13,0.0,0.0,3.58,90.0,1577835810,1577836799,1424,1
4,73838,51.344711,12.405900,16,2019-12-31 23:43:47,2019-12-31 23:59:59,0,25,1,881f1ab965fffff,2.13,0.0,0.0,3.58,90.0,1577835827,1577836799,1424,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883316,71996,51.332053,12.403324,130,2020-12-30 20:41:15,2020-12-30 22:51:54,0,0,0,881f1a165bfffff,2.61,0.0,0.0,3.58,82.0,1609360875,1609368714,1241,2
883317,93372,51.320791,12.405072,52,2020-12-30 20:46:14,2020-12-30 21:38:59,0,0,0,881f1a1657fffff,2.61,0.0,0.0,3.58,82.0,1609361174,1609364339,1246,2
883318,41797,51.331062,12.390168,24,2020-12-30 20:48:59,2020-12-30 21:13:09,0,0,0,881f1a165dfffff,2.61,0.0,0.0,3.58,82.0,1609361339,1609362789,1249,2
883319,93645,51.340196,12.402059,88,2020-12-30 21:00:57,2020-12-30 22:29:07,0,0,0,881f1ab965fffff,2.76,0.0,0.0,4.47,82.0,1609362057,1609367347,1261,2


# Add month feature

In [31]:
def add_month_feature(row):
    return row['time_start'].strftime("%m")

df_points['month'] = df_points.apply(add_month_feature,axis=1)

In [32]:
#del df_points['time_start']
#del df_points['time_end']

Encode Hex_id

from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df_points['hex_enc']= label_encoder.fit_transform(df_points['hex_id'])

Sort DF

In [33]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night,hex_id,temp,rain,snow,wind_speed,humidity,dt_start,dt_end,start_min,day,month
0,93513,51.317649,12.373610,50,2019-12-31 23:09:34,2019-12-31 23:59:59,0,1439,1,881f1a1647fffff,2.13,0.0,0.0,3.58,90.0,1577833774,1577836799,1390,1,12
1,74459,51.320440,12.345386,27,2019-12-31 23:32:03,2019-12-31 23:59:59,0,98,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835123,1577836799,1412,1,12
2,75845,51.320418,12.345038,27,2019-12-31 23:32:48,2019-12-31 23:59:59,0,99,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835168,1577836799,1413,1,12
3,100227,51.339529,12.373269,16,2019-12-31 23:43:30,2019-12-31 23:59:59,0,133,1,881f1a8cb5fffff,2.13,0.0,0.0,3.58,90.0,1577835810,1577836799,1424,1,12
4,73838,51.344711,12.405900,16,2019-12-31 23:43:47,2019-12-31 23:59:59,0,25,1,881f1ab965fffff,2.13,0.0,0.0,3.58,90.0,1577835827,1577836799,1424,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883316,71996,51.332053,12.403324,130,2020-12-30 20:41:15,2020-12-30 22:51:54,0,0,0,881f1a165bfffff,2.61,0.0,0.0,3.58,82.0,1609360875,1609368714,1241,2,12
883317,93372,51.320791,12.405072,52,2020-12-30 20:46:14,2020-12-30 21:38:59,0,0,0,881f1a1657fffff,2.61,0.0,0.0,3.58,82.0,1609361174,1609364339,1246,2,12
883318,41797,51.331062,12.390168,24,2020-12-30 20:48:59,2020-12-30 21:13:09,0,0,0,881f1a165dfffff,2.61,0.0,0.0,3.58,82.0,1609361339,1609362789,1249,2,12
883319,93645,51.340196,12.402059,88,2020-12-30 21:00:57,2020-12-30 22:29:07,0,0,0,881f1ab965fffff,2.76,0.0,0.0,4.47,82.0,1609362057,1609367347,1261,2,12


df_points = df_points.reindex(columns=['bike_id', 'lat', 'lng', 'dt_start', 'dt_end', 'hex_id', 'temp', 'rain', 'snow', 'wind_speed', 'humidity', 'month', 'day', 'start_min', 'over_night', 'idle_time_next_day', 'idle_time'])


df_points.sort_values(by=['dt_start'])

In [34]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night,hex_id,temp,rain,snow,wind_speed,humidity,dt_start,dt_end,start_min,day,month
0,93513,51.317649,12.373610,50,2019-12-31 23:09:34,2019-12-31 23:59:59,0,1439,1,881f1a1647fffff,2.13,0.0,0.0,3.58,90.0,1577833774,1577836799,1390,1,12
1,74459,51.320440,12.345386,27,2019-12-31 23:32:03,2019-12-31 23:59:59,0,98,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835123,1577836799,1412,1,12
2,75845,51.320418,12.345038,27,2019-12-31 23:32:48,2019-12-31 23:59:59,0,99,1,881f1a8d93fffff,2.13,0.0,0.0,3.58,90.0,1577835168,1577836799,1413,1,12
3,100227,51.339529,12.373269,16,2019-12-31 23:43:30,2019-12-31 23:59:59,0,133,1,881f1a8cb5fffff,2.13,0.0,0.0,3.58,90.0,1577835810,1577836799,1424,1,12
4,73838,51.344711,12.405900,16,2019-12-31 23:43:47,2019-12-31 23:59:59,0,25,1,881f1ab965fffff,2.13,0.0,0.0,3.58,90.0,1577835827,1577836799,1424,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883316,71996,51.332053,12.403324,130,2020-12-30 20:41:15,2020-12-30 22:51:54,0,0,0,881f1a165bfffff,2.61,0.0,0.0,3.58,82.0,1609360875,1609368714,1241,2,12
883317,93372,51.320791,12.405072,52,2020-12-30 20:46:14,2020-12-30 21:38:59,0,0,0,881f1a1657fffff,2.61,0.0,0.0,3.58,82.0,1609361174,1609364339,1246,2,12
883318,41797,51.331062,12.390168,24,2020-12-30 20:48:59,2020-12-30 21:13:09,0,0,0,881f1a165dfffff,2.61,0.0,0.0,3.58,82.0,1609361339,1609362789,1249,2,12
883319,93645,51.340196,12.402059,88,2020-12-30 21:00:57,2020-12-30 22:29:07,0,0,0,881f1ab965fffff,2.76,0.0,0.0,4.47,82.0,1609362057,1609367347,1261,2,12


# Save

df_points.to_csv('../data/df_points/df_points_2018.csv', index=False)

import pandas as pd
df = pd.read_csv('../data/df_points/final_df_points_18_21.csv')
df = df.sort_values(by=['dt_start'])

# Add year Feature

In [35]:
def add_year_feature(row):
    return row['time_start'].year

df_points['year'] = df_points.apply(add_year_feature,axis=1)

# Add In_zone Feature

In [36]:
import os
import json
from shapely.geometry import shape, Point

flexzones_0 = []
flexzones_1 = []

def save_flexzones():
    # Flexzone 0 Euro
    directory_name_0 = f'../flexzones/0/'
    for file_name in os.listdir(directory_name_0):
        path = os.path.join(directory_name_0, file_name)
        if os.path.isfile(path):
           with open(path) as f:
                js = json.load(f)
                for feature in js['features']:
                    flexzones_0.append(feature)
    # Flexzone 1 Euro
    directory_name_1 = f'../flexzones/1/'
    for file_name in os.listdir(directory_name_1):
        path = os.path.join(directory_name_1, file_name)
        if os.path.isfile(path):
           with open(path) as f:
                js = json.load(f)
                for feature in js['features']:
                    flexzones_1.append(feature)


def point_in_polygons(row):
    y = row['lat']  # y = 51.331305
    x = row['lng']  # x = 12.344334

    # switch x and y-axis bc. scapely woks on x plane
    point = Point(x,y)

    for feature0 in flexzones_0:
        polygon = shape(feature0['geometry'])
        if polygon.contains(point):
            return feature0['properties']['name']

    for feature1 in flexzones_1:
        polygon = shape(feature1['geometry'])
        if polygon.contains(point):
            return feature1['properties']['name']
    else: return 0

In [37]:
save_flexzones()

In [38]:
df_points['zone_name'] = df_points.apply(point_in_polygons,axis=1)

In [39]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night,hex_id,...,snow,wind_speed,humidity,dt_start,dt_end,start_min,day,month,year,zone_name
0,93513,51.317649,12.373610,50,2019-12-31 23:09:34,2019-12-31 23:59:59,0,1439,1,881f1a1647fffff,...,0.0,3.58,90.0,1577833774,1577836799,1390,1,12,2019,Pinke Zone Leipzig Ost 1€
1,74459,51.320440,12.345386,27,2019-12-31 23:32:03,2019-12-31 23:59:59,0,98,1,881f1a8d93fffff,...,0.0,3.58,90.0,1577835123,1577836799,1412,1,12,2019,Pinke Zone Leipzig West 1€
2,75845,51.320418,12.345038,27,2019-12-31 23:32:48,2019-12-31 23:59:59,0,99,1,881f1a8d93fffff,...,0.0,3.58,90.0,1577835168,1577836799,1413,1,12,2019,Pinke Zone Leipzig West 1€
3,100227,51.339529,12.373269,16,2019-12-31 23:43:30,2019-12-31 23:59:59,0,133,1,881f1a8cb5fffff,...,0.0,3.58,90.0,1577835810,1577836799,1424,1,12,2019,Blaue Zone Leipzig Ost 0€
4,73838,51.344711,12.405900,16,2019-12-31 23:43:47,2019-12-31 23:59:59,0,25,1,881f1ab965fffff,...,0.0,3.58,90.0,1577835827,1577836799,1424,1,12,2019,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883316,71996,51.332053,12.403324,130,2020-12-30 20:41:15,2020-12-30 22:51:54,0,0,0,881f1a165bfffff,...,0.0,3.58,82.0,1609360875,1609368714,1241,2,12,2020,Pinke Zone Leipzig Ost 1€
883317,93372,51.320791,12.405072,52,2020-12-30 20:46:14,2020-12-30 21:38:59,0,0,0,881f1a1657fffff,...,0.0,3.58,82.0,1609361174,1609364339,1246,2,12,2020,0
883318,41797,51.331062,12.390168,24,2020-12-30 20:48:59,2020-12-30 21:13:09,0,0,0,881f1a165dfffff,...,0.0,3.58,82.0,1609361339,1609362789,1249,2,12,2020,Blaue Zone Leipzig Ost 0€
883319,93645,51.340196,12.402059,88,2020-12-30 21:00:57,2020-12-30 22:29:07,0,0,0,881f1ab965fffff,...,0.0,4.47,82.0,1609362057,1609367347,1261,2,12,2020,0


In [40]:
def in_zone(row):
    if row['zone_name'] != 0:
        return 1
    else: return 0

df_points['in_zone'] = df_points.apply(in_zone,axis=1)

In [ ]:
# add hex enc
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df_points['hex_enc'] = label_encoder.fit_transform(df_points['hex_id'])

# add on_station
def on_station(row):
    if row['end_station_number'] != 0:
        return 1
    else:
        return 0

df_points['on_station'] = df_points.apply(on_station, axis=1)

# add zone name_enc
zone_encoder = preprocessing.LabelEncoder()
df_points['zone_name_enc'] = zone_encoder.fit_transform(df_points['zone_name'])

In [41]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,end_station_number,idle_time_next_day,over_night,hex_id,...,wind_speed,humidity,dt_start,dt_end,start_min,day,month,year,zone_name,in_zone
0,93513,51.317649,12.373610,50,2019-12-31 23:09:34,2019-12-31 23:59:59,0,1439,1,881f1a1647fffff,...,3.58,90.0,1577833774,1577836799,1390,1,12,2019,Pinke Zone Leipzig Ost 1€,1
1,74459,51.320440,12.345386,27,2019-12-31 23:32:03,2019-12-31 23:59:59,0,98,1,881f1a8d93fffff,...,3.58,90.0,1577835123,1577836799,1412,1,12,2019,Pinke Zone Leipzig West 1€,1
2,75845,51.320418,12.345038,27,2019-12-31 23:32:48,2019-12-31 23:59:59,0,99,1,881f1a8d93fffff,...,3.58,90.0,1577835168,1577836799,1413,1,12,2019,Pinke Zone Leipzig West 1€,1
3,100227,51.339529,12.373269,16,2019-12-31 23:43:30,2019-12-31 23:59:59,0,133,1,881f1a8cb5fffff,...,3.58,90.0,1577835810,1577836799,1424,1,12,2019,Blaue Zone Leipzig Ost 0€,1
4,73838,51.344711,12.405900,16,2019-12-31 23:43:47,2019-12-31 23:59:59,0,25,1,881f1ab965fffff,...,3.58,90.0,1577835827,1577836799,1424,1,12,2019,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883316,71996,51.332053,12.403324,130,2020-12-30 20:41:15,2020-12-30 22:51:54,0,0,0,881f1a165bfffff,...,3.58,82.0,1609360875,1609368714,1241,2,12,2020,Pinke Zone Leipzig Ost 1€,1
883317,93372,51.320791,12.405072,52,2020-12-30 20:46:14,2020-12-30 21:38:59,0,0,0,881f1a1657fffff,...,3.58,82.0,1609361174,1609364339,1246,2,12,2020,0,0
883318,41797,51.331062,12.390168,24,2020-12-30 20:48:59,2020-12-30 21:13:09,0,0,0,881f1a165dfffff,...,3.58,82.0,1609361339,1609362789,1249,2,12,2020,Blaue Zone Leipzig Ost 0€,1
883319,93645,51.340196,12.402059,88,2020-12-30 21:00:57,2020-12-30 22:29:07,0,0,0,881f1ab965fffff,...,4.47,82.0,1609362057,1609367347,1261,2,12,2020,0,0


In [42]:
df_points = df_points.reindex(columns=['bike_id', 'lat', 'lng', 'dt_start', 'dt_end', 'hex_id','hex_enc', 'in_zone', 'zone_name','zone_name_enc','end_station_number','on_station','temp', 'rain', 'snow', 'wind_speed', 'humidity', 'year','month', 'day', 'start_min', 'over_night', 'idle_time_next_day', 'idle_time'])

df_points = df_points.sort_values(by=['dt_start'])
df_points

,bike_id,lat,lng,dt_start,dt_end,hex_id,in_zone,zone_name,end_station_number,temp,...,snow,wind_speed,humidity,year,month,day,start_min,over_night,idle_time_next_day,idle_time
0,93513,51.317649,12.373610,1577833774,1577836799,881f1a1647fffff,1,Pinke Zone Leipzig Ost 1€,0,2.13,...,0.0,3.58,90.0,2019,12,1,1390,1,1439,50
1,74459,51.320440,12.345386,1577835123,1577836799,881f1a8d93fffff,1,Pinke Zone Leipzig West 1€,0,2.13,...,0.0,3.58,90.0,2019,12,1,1412,1,98,27
2,75845,51.320418,12.345038,1577835168,1577836799,881f1a8d93fffff,1,Pinke Zone Leipzig West 1€,0,2.13,...,0.0,3.58,90.0,2019,12,1,1413,1,99,27
3,100227,51.339529,12.373269,1577835810,1577836799,881f1a8cb5fffff,1,Blaue Zone Leipzig Ost 0€,0,2.13,...,0.0,3.58,90.0,2019,12,1,1424,1,133,16
4,73838,51.344711,12.405900,1577835827,1577836799,881f1ab965fffff,0,0,0,2.13,...,0.0,3.58,90.0,2019,12,1,1424,1,25,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883316,71996,51.332053,12.403324,1609360875,1609368714,881f1a165bfffff,1,Pinke Zone Leipzig Ost 1€,0,2.61,...,0.0,3.58,82.0,2020,12,2,1241,0,0,130
883317,93372,51.320791,12.405072,1609361174,1609364339,881f1a1657fffff,0,0,0,2.61,...,0.0,3.58,82.0,2020,12,2,1246,0,0,52
883318,41797,51.331062,12.390168,1609361339,1609362789,881f1a165dfffff,1,Blaue Zone Leipzig Ost 0€,0,2.61,...,0.0,3.58,82.0,2020,12,2,1249,0,0,24
883319,93645,51.340196,12.402059,1609362057,1609367347,881f1ab965fffff,0,0,0,2.76,...,0.0,4.47,82.0,2020,12,2,1261,0,0,88


# SAVE

In [43]:
dur = time.time() - ts1
print(dur)
ty_res = time.gmtime(dur)
res = time.strftime("%H:%M:%S",ty_res)
print(res)

25273.7381439209
07:01:13


In [44]:
df_points.to_csv('../data/df_points/df_points_2020_v2.csv', index=False)